# CNN

In [3]:
import os
import random
import numpy as np
import pandas as pd
import optuna
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ==========================================
# 1) CONFIGURATION (NO TEST LEAKAGE)
# ==========================================
TIME_WINDOW = "15min"
FILE_PATH   = "../data/derivative_data_2025.parquet"
N_TRIALS    = 20

SEQ_LEN     = 16
MAX_EPOCHS  = 60
PATIENCE    = 10
RANDOM_SEED = 42
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

VAL_FRAC = 0.10  # validation slice from the end of the training block

def seed_everything(seed=RANDOM_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(RANDOM_SEED)

# ==========================================
# 2) DATA LOADING (SAME LOGIC)
# ==========================================
print(">>> Loading Parquet Data...")
df = pd.read_parquet(FILE_PATH, engine="pyarrow")

if pd.api.types.is_numeric_dtype(df["timestamp"]):
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
else:
    df["timestamp"] = pd.to_datetime(df["timestamp"])

df = df[df["instrument_name"].str.contains("BTC")]

split_names = df["instrument_name"].str.split("-", expand=True)
df["Expiry_Str"] = split_names[1]
df["Strike"]     = split_names[2].astype(float)
df["Type"]       = split_names[3]
df["Expiry"]     = pd.to_datetime(df["Expiry_Str"], format="%d%b%y", errors="coerce")

df["days_to_expiry"] = (df["Expiry"] - df["timestamp"]).dt.total_seconds() / (24 * 3600)
df["moneyness"]      = df["index_price"] / df["Strike"]
df["iv"]             = pd.to_numeric(df["iv"], errors="coerce")
df = df[df["iv"] > 0]

# ==========================================
# 3) FEATURE ENGINEERING (SAME LOGIC)
# ==========================================
print(f">>> Resampling to {TIME_WINDOW} bars...")

def calculate_features(sub_df: pd.DataFrame):
    if sub_df.empty:
        return None
    stats = {}

    atm_mask = sub_df["moneyness"].between(0.98, 1.02)
    if atm_mask.any():
        stats["ATM_IV"] = sub_df.loc[atm_mask, "iv"].mean()
    else:
        stats["ATM_IV"] = sub_df["iv"].mean()

    put_iv  = sub_df.loc[sub_df["Type"] == "P", "iv"].mean()
    call_iv = sub_df.loc[sub_df["Type"] == "C", "iv"].mean()
    put_iv  = put_iv  if pd.notna(put_iv)  else stats["ATM_IV"]
    call_iv = call_iv if pd.notna(call_iv) else stats["ATM_IV"]
    stats["Skew"] = put_iv - call_iv

    near_mask = sub_df["days_to_expiry"] <= 10
    far_mask  = sub_df["days_to_expiry"] > 10
    iv_near = sub_df.loc[near_mask, "iv"].mean()
    iv_far  = sub_df.loc[far_mask,  "iv"].mean()
    iv_near = iv_near if (pd.notna(iv_near) and iv_near > 0) else stats["ATM_IV"]
    iv_far  = iv_far  if pd.notna(iv_far) else stats["ATM_IV"]
    stats["Term_Structure"] = iv_far / iv_near

    stats["Close_Price"] = sub_df["index_price"].iloc[-1]
    stats["Volume"]      = sub_df["amount"].sum()

    return pd.Series(stats)

df_grouped = df.set_index("timestamp").resample(TIME_WINDOW).apply(calculate_features)
df_grouped = df_grouped.ffill()

# ==========================================
# 4) TARGET CREATION (SAME LOGIC)
# ==========================================
print(">>> Creating Delta Target...")

df_grouped["ATM_IV_Change"]      = df_grouped["ATM_IV"].diff()
df_grouped["ATM_IV_Change_Lag1"] = df_grouped["ATM_IV_Change"].shift(1)

df_grouped["returns"]      = np.log(df_grouped["Close_Price"] / df_grouped["Close_Price"].shift(1))
df_grouped["Realized_Vol"] = df_grouped["returns"].shift(1).rolling(window=4).std()

df_grouped["Target_Delta"] = df_grouped["ATM_IV"].shift(-1) - df_grouped["ATM_IV"]

model_data = df_grouped.dropna()

feature_cols = ["ATM_IV", "Skew", "Term_Structure", "Realized_Vol",
                "Volume", "ATM_IV_Change", "ATM_IV_Change_Lag1"]

X = model_data[feature_cols]
y = model_data["Target_Delta"]

print(f"Data Prepared (tabular). Rows: {len(model_data)}")

# ==========================================
# 5) SEQUENCE BUILD (SAME LOGIC)
# ==========================================
def make_sequences(X_df: pd.DataFrame, y_s: pd.Series, seq_len: int):
    Xv = X_df.values.astype(np.float32)
    yv = y_s.values.astype(np.float32)

    X_seq, y_seq = [], []
    for t in range(seq_len - 1, len(Xv)):
        X_seq.append(Xv[t - seq_len + 1 : t + 1, :])
        y_seq.append(yv[t])
    return np.stack(X_seq), np.array(y_seq)

X_seq, y_seq = make_sequences(X, y, SEQ_LEN)
n_samples, _, n_features = X_seq.shape
print(f"NN Sequences: X={X_seq.shape}, y={y_seq.shape}")

# ==========================================
# 5.5) SPLIT (FIXED: TRAIN/VAL/TEST CHRONOLOGICAL)
# ==========================================
split_idx = int(n_samples * 0.8)  # test starts here

X_train_full, y_train_full = X_seq[:split_idx], y_seq[:split_idx]
X_test,       y_test       = X_seq[split_idx:], y_seq[split_idx:]

val_size  = max(1, int(len(X_train_full) * VAL_FRAC))
val_start = len(X_train_full) - val_size

X_train, y_train = X_train_full[:val_start], y_train_full[:val_start]
X_val,   y_val   = X_train_full[val_start:], y_train_full[val_start:]

print(f"NN Split: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")

# ==========================================
# 6) TORCH HELPERS
# ==========================================
class SeqDataset(Dataset):
    def __init__(self, X_seq, y_seq):
        self.X = torch.from_numpy(X_seq)
        self.y = torch.from_numpy(y_seq).unsqueeze(-1)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def fit_scaler_on_train(X_train_seq):
    scaler = StandardScaler()
    N, T, F = X_train_seq.shape
    scaler.fit(X_train_seq.reshape(N * T, F))
    return scaler

def apply_scaler(scaler, X_seq):
    N, T, F = X_seq.shape
    return scaler.transform(X_seq.reshape(N * T, F)).reshape(N, T, F).astype(np.float32)

def rmse_np(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

@torch.no_grad()
def predict_model(model, X_seq_np, batch_size=1024):
    model.eval()
    ds = SeqDataset(X_seq_np, np.zeros((len(X_seq_np),), dtype=np.float32))
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False)
    preds = []
    for xb, _ in dl:
        xb = xb.to(DEVICE)
        preds.append(model(xb).detach().cpu().numpy().reshape(-1))
    return np.concatenate(preds)

def train_with_early_stopping(model, train_loader, val_loader, lr, weight_decay=0.0,
                              max_epochs=MAX_EPOCHS, patience=PATIENCE):
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = nn.MSELoss()

    best_rmse = np.inf
    best_state = None
    bad_epochs = 0

    for _epoch in range(1, max_epochs + 1):
        model.train()
        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)
            yhat = model(xb)
            loss = loss_fn(yhat, yb)
            loss.backward()
            opt.step()

        # val RMSE
        model.eval()
        vpred, vtrue = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE)
                yb = yb.to(DEVICE)
                yhat = model(xb)
                vpred.append(yhat.detach().cpu().numpy().reshape(-1))
                vtrue.append(yb.detach().cpu().numpy().reshape(-1))
        vpred = np.concatenate(vpred)
        vtrue = np.concatenate(vtrue)
        val_rmse = rmse_np(vtrue, vpred)

        if val_rmse < best_rmse - 1e-8:
            best_rmse = val_rmse
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return best_rmse

# ==========================================
# 7) MODELS
# ==========================================
class RNNRegressor(nn.Module):
    def __init__(self, n_features, hidden_size, num_layers, dropout, rnn_type="GRU"):
        super().__init__()
        rnn_cls = nn.GRU if rnn_type.upper() == "GRU" else nn.LSTM
        self.rnn = rnn_cls(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=(dropout if num_layers > 1 else 0.0),
        )
        self.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(hidden_size, 1))

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.head(out[:, -1, :])

class CNN1DRegressor(nn.Module):
    def __init__(self, n_features, channels, kernel_size, dropout):
        super().__init__()
        padding = kernel_size // 2
        self.net = nn.Sequential(
            nn.Conv1d(n_features, channels, kernel_size, padding=padding),
            nn.ReLU(),
            nn.Conv1d(channels, channels, kernel_size, padding=padding),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )
        self.head = nn.Sequential(nn.Flatten(), nn.Dropout(dropout), nn.Linear(channels, 1))

    def forward(self, x):
        x = x.transpose(1, 2)
        return self.head(self.net(x))

# ==========================================
# 8) OPTUNA TUNING (NO TEST LEAKAGE)
#   - CV only uses X_train_full (train+val block)
# ==========================================
print("\n>>> Tuning RNN Hyperparameters...")

def objective_rnn(trial):
    seed_everything(RANDOM_SEED)

    rnn_type    = trial.suggest_categorical("rnn_type", ["GRU", "LSTM"])
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    num_layers  = trial.suggest_int("num_layers", 1, 3)
    dropout     = trial.suggest_float("dropout", 0.0, 0.4)
    lr          = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    wd          = trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)
    batch_size  = trial.suggest_categorical("batch_size", [64, 128, 256])

    tscv = TimeSeriesSplit(n_splits=3)
    scores = []

    for tr_idx, va_idx in tscv.split(X_train_full):
        X_tr, y_tr = X_train_full[tr_idx], y_train_full[tr_idx]
        X_va, y_va = X_train_full[va_idx], y_train_full[va_idx]

        scaler = fit_scaler_on_train(X_tr)
        X_tr_s = apply_scaler(scaler, X_tr)
        X_va_s = apply_scaler(scaler, X_va)

        train_loader = DataLoader(SeqDataset(X_tr_s, y_tr), batch_size=batch_size, shuffle=False)
        val_loader   = DataLoader(SeqDataset(X_va_s, y_va), batch_size=batch_size, shuffle=False)

        model = RNNRegressor(n_features, hidden_size, num_layers, dropout, rnn_type=rnn_type).to(DEVICE)
        scores.append(train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd))

    return float(np.mean(scores))

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_rnn = optuna.create_study(direction="minimize")
study_rnn.optimize(objective_rnn, n_trials=N_TRIALS)
print(f"Best RNN Params: {study_rnn.best_params}")

print("\n>>> Tuning CNN Hyperparameters...")

def objective_cnn(trial):
    seed_everything(RANDOM_SEED)

    channels    = trial.suggest_int("channels", 16, 128)
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5, 7])
    dropout     = trial.suggest_float("dropout", 0.0, 0.4)
    lr          = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    wd          = trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)
    batch_size  = trial.suggest_categorical("batch_size", [64, 128, 256])

    tscv = TimeSeriesSplit(n_splits=3)
    scores = []

    for tr_idx, va_idx in tscv.split(X_train_full):
        X_tr, y_tr = X_train_full[tr_idx], y_train_full[tr_idx]
        X_va, y_va = X_train_full[va_idx], y_train_full[va_idx]

        scaler = fit_scaler_on_train(X_tr)
        X_tr_s = apply_scaler(scaler, X_tr)
        X_va_s = apply_scaler(scaler, X_va)

        train_loader = DataLoader(SeqDataset(X_tr_s, y_tr), batch_size=batch_size, shuffle=False)
        val_loader   = DataLoader(SeqDataset(X_va_s, y_va), batch_size=batch_size, shuffle=False)

        model = CNN1DRegressor(n_features, channels, kernel_size, dropout).to(DEVICE)
        scores.append(train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd))

    return float(np.mean(scores))

study_cnn = optuna.create_study(direction="minimize")
study_cnn.optimize(objective_cnn, n_trials=N_TRIALS)
print(f"Best CNN Params: {study_cnn.best_params}")

# ==========================================
# 9) FINAL TRAINING & EVALUATION (FIXED)
#   - Fit scaler on TRAIN only
#   - Early stop on VAL only
#   - Test untouched until final metrics
# ==========================================
def evaluate_final(model_name, build_model, best_params):
    print(f"\n>>> Training Final {model_name}...")

    batch_size = int(best_params.get("batch_size", 128))
    lr = float(best_params["lr"])
    wd = float(best_params["weight_decay"])

    # scaler on TRAIN only
    scaler = fit_scaler_on_train(X_train)
    X_train_s = apply_scaler(scaler, X_train)
    X_val_s   = apply_scaler(scaler, X_val)
    X_test_s  = apply_scaler(scaler, X_test)

    train_loader = DataLoader(SeqDataset(X_train_s, y_train), batch_size=batch_size, shuffle=False)
    val_loader   = DataLoader(SeqDataset(X_val_s,   y_val),   batch_size=batch_size, shuffle=False)

    model = build_model().to(DEVICE)
    _ = train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd)

    # test untouched until here
    y_pred = predict_model(model, X_test_s)

    final_rmse = rmse_np(y_test, y_pred)
    naive_rmse = rmse_np(y_test, np.zeros_like(y_test))

    pred_signs   = np.sign(y_pred)
    actual_signs = np.sign(y_test)
    valid_idx = actual_signs != 0
    dir_acc = accuracy_score(actual_signs[valid_idx], pred_signs[valid_idx])

    print(f"\n=== {model_name} RESULTS ===")
    print(f"Model RMSE:   {final_rmse:.6f}")
    print(f"Baseline RMSE:{naive_rmse:.6f}")
    print(f"Improvement:  {((naive_rmse - final_rmse)/naive_rmse)*100:.2f}%")
    print(f"Directional Accuracy: {dir_acc*100:.2f}%")

    plt.figure(figsize=(12, 6))
    plt.plot(y_test[:100], label="Actual Change", alpha=0.7)
    plt.plot(y_pred[:100], label=f"{model_name} Pred", linestyle="--")
    plt.axhline(0, lw=0.5)
    plt.title(f"{model_name}: ATM IV Change Prediction (Delta)")
    plt.legend()
    plt.show()

    return final_rmse, dir_acc

# ---- Final RNN ----
bestRNN = study_rnn.best_params

def make_final_rnn():
    return RNNRegressor(
        n_features=n_features,
        hidden_size=int(bestRNN["hidden_size"]),
        num_layers=int(bestRNN["num_layers"]),
        dropout=float(bestRNN["dropout"]),
        rnn_type=str(bestRNN["rnn_type"])
    )

rmse_rnn, dir_rnn = evaluate_final("RNN", make_final_rnn, bestRNN)

# ---- Final CNN ----
bestCNN = study_cnn.best_params

def make_final_cnn():
    return CNN1DRegressor(
        n_features=n_features,
        channels=int(bestCNN["channels"]),
        kernel_size=int(bestCNN["kernel_size"]),
        dropout=float(bestCNN["dropout"])
    )

rmse_cnn, dir_cnn = evaluate_final("CNN", make_final_cnn, bestCNN)

print("\n>>> Summary")
print(f"RNN  RMSE={rmse_rnn:.6f} | DirAcc={dir_rnn*100:.2f}%")
print(f"CNN  RMSE={rmse_cnn:.6f} | DirAcc={dir_cnn*100:.2f}%")


>>> Loading Parquet Data...
>>> Resampling to 15min bars...
>>> Creating Delta Target...
Data Prepared (tabular). Rows: 35130
NN Sequences: X=(35115, 16, 7), y=(35115,)
NN Split: train=25283, val=2809, test=7023

>>> Tuning RNN Hyperparameters...


[W 2026-01-14 20:19:49,729] Trial 15 failed with parameters: {'rnn_type': 'LSTM', 'hidden_size': 76, 'num_layers': 2, 'dropout': 0.23338622430617414, 'lr': 0.0015294488790853964, 'weight_decay': 2.8842752469673902e-06, 'batch_size': 128} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/shah/CODE_BOOK_3/MLII/MLFinalProject/.venv/lib/python3.13/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/rz/hb914cgn7wb_sxdhbbm8krnh0000gn/T/ipykernel_71391/1963971102.py", line 311, in objective_rnn
    scores.append(train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd))
                  ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/rz/hb914cgn7wb_sxdhbbm8krnh0000gn/T/ipykernel_71391/1963971102.py", line 222, in train_with_early_stopping
    yhat = model(xb)
  File "/Users/shah/CODE_BOOK_3/M

KeyboardInterrupt: 

## CNN RNN

In [ ]:
import os, random
import numpy as np
import pandas as pd
import optuna
import matplotlib.pyplot as plt
from tqdm import tqdm


from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ==========================================
# 0) CONFIG
# ==========================================
TIME_WINDOW = "15min"
FILE_PATH   = "../data/derivative_data_2025.parquet"
N_TRIALS    = 20

SEQ_LEN     = 16
MAX_EPOCHS  = 60
PATIENCE    = 10
RANDOM_SEED = 42
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
VAL_FRAC    = 0.10  # val slice from end of training block

def seed_everything(seed=RANDOM_SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(RANDOM_SEED)

# ==========================================
# 1) DATA LOADING
# ==========================================
print(">>> Loading Parquet Data...")
df = pd.read_parquet(FILE_PATH, engine="pyarrow")

if pd.api.types.is_numeric_dtype(df["timestamp"]):
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
else:
    df["timestamp"] = pd.to_datetime(df["timestamp"])

df = df[df["instrument_name"].str.contains("BTC")]

split_names = df["instrument_name"].str.split("-", expand=True)
df["Expiry_Str"] = split_names[1]
df["Strike"]     = split_names[2].astype(float)
df["Type"]       = split_names[3]
df["Expiry"]     = pd.to_datetime(df["Expiry_Str"], format="%d%b%y", errors="coerce")

df["days_to_expiry"] = (df["Expiry"] - df["timestamp"]).dt.total_seconds() / (24 * 3600)
df["moneyness"]      = df["index_price"] / df["Strike"]
df["iv"]             = pd.to_numeric(df["iv"], errors="coerce")
df = df[df["iv"] > 0]

# ==========================================
# 2) FEATURE ENGINEERING (15-min bars)
# ==========================================
print(f">>> Resampling to {TIME_WINDOW} bars...")

def calculate_features(sub_df: pd.DataFrame):
    if sub_df.empty:
        return None
    stats = {}

    atm_mask = sub_df["moneyness"].between(0.98, 1.02)
    stats["ATM_IV"] = sub_df.loc[atm_mask, "iv"].mean() if atm_mask.any() else sub_df["iv"].mean()

    put_iv  = sub_df.loc[sub_df["Type"] == "P", "iv"].mean()
    call_iv = sub_df.loc[sub_df["Type"] == "C", "iv"].mean()
    put_iv  = put_iv  if pd.notna(put_iv)  else stats["ATM_IV"]
    call_iv = call_iv if pd.notna(call_iv) else stats["ATM_IV"]
    stats["Skew"] = put_iv - call_iv

    near_mask = sub_df["days_to_expiry"] <= 10
    far_mask  = sub_df["days_to_expiry"] > 10
    iv_near = sub_df.loc[near_mask, "iv"].mean()
    iv_far  = sub_df.loc[far_mask,  "iv"].mean()
    iv_near = iv_near if (pd.notna(iv_near) and iv_near > 0) else stats["ATM_IV"]
    iv_far  = iv_far  if pd.notna(iv_far) else stats["ATM_IV"]
    stats["Term_Structure"] = iv_far / iv_near

    stats["Close_Price"] = sub_df["index_price"].iloc[-1]
    stats["Volume"]      = sub_df["amount"].sum()
    return pd.Series(stats)

df_grouped = df.set_index("timestamp").resample(TIME_WINDOW).apply(calculate_features)
df_grouped = df_grouped.ffill()

# --- Option B: time-of-day + day-of-week cyclical encodings ---
ts = df_grouped.index
tod = ts.hour + ts.minute / 60.0
df_grouped["tod_sin"] = np.sin(2 * np.pi * tod / 24.0)
df_grouped["tod_cos"] = np.cos(2 * np.pi * tod / 24.0)

dow = ts.dayofweek.astype(float)  # 0..6
df_grouped["dow_sin"] = np.sin(2 * np.pi * dow / 7.0)
df_grouped["dow_cos"] = np.cos(2 * np.pi * dow / 7.0)

# ==========================================
# 3) TARGET CREATION (same as LGBM/CatBoost)
# ==========================================
print(">>> Creating Delta Target...")

df_grouped["ATM_IV_Change"]      = df_grouped["ATM_IV"].diff()
df_grouped["ATM_IV_Change_Lag1"] = df_grouped["ATM_IV_Change"].shift(1)

df_grouped["returns"]      = np.log(df_grouped["Close_Price"] / df_grouped["Close_Price"].shift(1))
df_grouped["Realized_Vol"] = df_grouped["returns"].shift(1).rolling(window=4).std()

df_grouped["Target_Delta"] = df_grouped["ATM_IV"].shift(-1) - df_grouped["ATM_IV"]

model_data = df_grouped.dropna()

feature_cols = [
    "ATM_IV","Skew","Term_Structure","Realized_Vol","Volume","ATM_IV_Change","ATM_IV_Change_Lag1",
    "tod_sin","tod_cos","dow_sin","dow_cos"
]

X = model_data[feature_cols]
y = model_data["Target_Delta"]

print(f"Data Prepared (tabular). Rows: {len(model_data)} | Features: {len(feature_cols)}")

# ==========================================
# 4) SEQUENCES
# ==========================================
def make_sequences(X_df: pd.DataFrame, y_s: pd.Series, seq_len: int):
    Xv = X_df.values.astype(np.float32)
    yv = y_s.values.astype(np.float32)

    X_seq, y_seq = [], []
    for t in range(seq_len - 1, len(Xv)):
        X_seq.append(Xv[t - seq_len + 1 : t + 1, :])
        y_seq.append(yv[t])
    return np.stack(X_seq), np.array(y_seq)

X_seq, y_seq = make_sequences(X, y, SEQ_LEN)
n_samples, _, n_features = X_seq.shape
print(f"NN Sequences: X={X_seq.shape}, y={y_seq.shape}")

# ==========================================
# 5) SPLIT (train/val/test chronological; no leakage)
# ==========================================
split_idx = int(n_samples * 0.8)  # test starts here
X_train_full, y_train_full = X_seq[:split_idx], y_seq[:split_idx]
X_test,       y_test       = X_seq[split_idx:], y_seq[split_idx:]

val_size  = max(1, int(len(X_train_full) * VAL_FRAC))
val_start = len(X_train_full) - val_size

X_train, y_train = X_train_full[:val_start], y_train_full[:val_start]
X_val,   y_val   = X_train_full[val_start:], y_train_full[val_start:]

print(f"NN Split: train={len(X_train)}, val={len(X_val)}, test={len(X_test)}")

# ==========================================
# 6) TORCH HELPERS
# ==========================================
class SeqDataset(Dataset):
    def __init__(self, X_seq, y_seq):
        self.X = torch.from_numpy(X_seq)                      # (N,T,F)
        self.y = torch.from_numpy(y_seq).unsqueeze(-1)        # (N,1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

def fit_scaler_on_train(X_train_seq):
    scaler = StandardScaler()
    N, T, F = X_train_seq.shape
    scaler.fit(X_train_seq.reshape(N*T, F))
    return scaler

def apply_scaler(scaler, X_seq):
    N, T, F = X_seq.shape
    return scaler.transform(X_seq.reshape(N*T, F)).reshape(N, T, F).astype(np.float32)

def rmse_np(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

@torch.no_grad()
def predict_model(model, X_seq_np, batch_size=1024):
    model.eval()
    dl = DataLoader(SeqDataset(X_seq_np, np.zeros((len(X_seq_np),), dtype=np.float32)),
                    batch_size=batch_size, shuffle=False)
    preds = []
    for xb, _ in dl:
        xb = xb.to(DEVICE)
        preds.append(model(xb).detach().cpu().numpy().reshape(-1))
    return np.concatenate(preds)

def train_with_early_stopping(model, train_loader, val_loader, lr, weight_decay=0.0,
                              max_epochs=MAX_EPOCHS, patience=PATIENCE):
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    loss_fn = nn.MSELoss()

    best_rmse = np.inf
    best_state = None
    bad_epochs = 0

    # for _epoch in range(1, max_epochs + 1):
    for _epoch in tqdm(range(1, max_epochs + 1), desc="epochs", leave=False):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad(set_to_none=True)
            yhat = model(xb)
            loss = loss_fn(yhat, yb)
            loss.backward()
            opt.step()

        # val RMSE
        model.eval()
        vpred, vtrue = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                yhat = model(xb)
                vpred.append(yhat.detach().cpu().numpy().reshape(-1))
                vtrue.append(yb.detach().cpu().numpy().reshape(-1))
        vpred = np.concatenate(vpred); vtrue = np.concatenate(vtrue)
        val_rmse = rmse_np(vtrue, vpred)

        if val_rmse < best_rmse - 1e-8:
            best_rmse = val_rmse
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= patience:
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    return best_rmse

# ==========================================
# 7) MODELS
# ==========================================
class RNNRegressor(nn.Module):
    def __init__(self, n_features, hidden_size, num_layers, dropout, rnn_type="GRU"):
        super().__init__()
        rnn_cls = nn.GRU if rnn_type.upper() == "GRU" else nn.LSTM
        self.rnn = rnn_cls(
            input_size=n_features,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=(dropout if num_layers > 1 else 0.0),
        )
        self.head = nn.Sequential(nn.Dropout(dropout), nn.Linear(hidden_size, 1))
    def forward(self, x):
        out, _ = self.rnn(x)
        return self.head(out[:, -1, :])

class CNN1DRegressor(nn.Module):
    def __init__(self, n_features, channels, kernel_size, dropout):
        super().__init__()
        padding = kernel_size // 2
        self.net = nn.Sequential(
            nn.Conv1d(n_features, channels, kernel_size, padding=padding),
            nn.ReLU(),
            nn.Conv1d(channels, channels, kernel_size, padding=padding),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )
        self.head = nn.Sequential(nn.Flatten(), nn.Dropout(dropout), nn.Linear(channels, 1))
    def forward(self, x):
        x = x.transpose(1, 2)  # (N,F,T)
        return self.head(self.net(x))

# ==========================================
# 8) OPTUNA TUNING (CV only on train_full; test untouched)
# ==========================================
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("\n>>> Tuning RNN Hyperparameters...")

def objective_rnn(trial):
    seed_everything(RANDOM_SEED)

    rnn_type    = trial.suggest_categorical("rnn_type", ["GRU", "LSTM"])
    hidden_size = trial.suggest_int("hidden_size", 16, 128)
    num_layers  = trial.suggest_int("num_layers", 1, 3)
    dropout     = trial.suggest_float("dropout", 0.0, 0.4)
    lr          = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    wd          = trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)
    batch_size  = trial.suggest_categorical("batch_size", [64, 128, 256])

    tscv = TimeSeriesSplit(n_splits=3)
    scores = []

    for tr_idx, va_idx in tscv.split(X_train_full):
        X_tr, y_tr = X_train_full[tr_idx], y_train_full[tr_idx]
        X_va, y_va = X_train_full[va_idx], y_train_full[va_idx]

        scaler = fit_scaler_on_train(X_tr)
        X_tr_s = apply_scaler(scaler, X_tr)
        X_va_s = apply_scaler(scaler, X_va)

        train_loader = DataLoader(SeqDataset(X_tr_s, y_tr), batch_size=batch_size, shuffle=False)
        val_loader   = DataLoader(SeqDataset(X_va_s, y_va), batch_size=batch_size, shuffle=False)

        model = RNNRegressor(n_features, hidden_size, num_layers, dropout, rnn_type=rnn_type).to(DEVICE)
        scores.append(train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd))

    return float(np.mean(scores))

study_rnn = optuna.create_study(direction="minimize")
study_rnn.optimize(objective_rnn, n_trials=N_TRIALS)
print(f"Best RNN Params: {study_rnn.best_params}")

print("\n>>> Tuning CNN Hyperparameters...")

def objective_cnn(trial):
    seed_everything(RANDOM_SEED)

    channels    = trial.suggest_int("channels", 16, 128)
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5, 7])
    dropout     = trial.suggest_float("dropout", 0.0, 0.4)
    lr          = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    wd          = trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)
    batch_size  = trial.suggest_categorical("batch_size", [64, 128, 256])

    tscv = TimeSeriesSplit(n_splits=3)
    scores = []

    for tr_idx, va_idx in tscv.split(X_train_full):
        X_tr, y_tr = X_train_full[tr_idx], y_train_full[tr_idx]
        X_va, y_va = X_train_full[va_idx], y_train_full[va_idx]

        scaler = fit_scaler_on_train(X_tr)
        X_tr_s = apply_scaler(scaler, X_tr)
        X_va_s = apply_scaler(scaler, X_va)

        train_loader = DataLoader(SeqDataset(X_tr_s, y_tr), batch_size=batch_size, shuffle=False)
        val_loader   = DataLoader(SeqDataset(X_va_s, y_va), batch_size=batch_size, shuffle=False)

        model = CNN1DRegressor(n_features, channels, kernel_size, dropout).to(DEVICE)
        scores.append(train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd))

    return float(np.mean(scores))

study_cnn = optuna.create_study(direction="minimize")
study_cnn.optimize(objective_cnn, n_trials=N_TRIALS)
print(f"Best CNN Params: {study_cnn.best_params}")

# ==========================================
# 9) PERMUTATION IMPORTANCE (NN analogue of "feature importance")
# ==========================================
def permutation_importance_seq(model, X_test_s, y_test, feature_names, batch_size=1024, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    base_pred = predict_model(model, X_test_s, batch_size=batch_size)
    base_rmse = rmse_np(y_test, base_pred)

    importances = []
    # for j, name in enumerate(feature_names):
    for j, name in tqdm(list(enumerate(feature_names)), desc="perm-imp", leave=False):
        Xp = X_test_s.copy()
        flat = Xp[:, :, j].reshape(-1)
        rng.shuffle(flat)
        Xp[:, :, j] = flat.reshape(Xp.shape[0], Xp.shape[1])
        pred_p = predict_model(model, Xp, batch_size=batch_size)
        rmse_p = rmse_np(y_test, pred_p)
        importances.append(rmse_p - base_rmse)  # increase in RMSE

    return base_rmse, np.array(importances, dtype=float)

def plot_feature_importance(importances, feature_names, title):
    idx = np.argsort(importances)
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(idx)), importances[idx], align="center")
    plt.yticks(range(len(idx)), np.array(feature_names)[idx])
    plt.title(title)
    plt.show()

# ==========================================
# 10) FINAL TRAINING & EVALUATION (no leakage)
# ==========================================
def evaluate_final(model_name, build_model, best_params):
    print(f"\n>>> Training Final {model_name}...")

    batch_size = int(best_params.get("batch_size", 128))
    lr = float(best_params["lr"])
    wd = float(best_params["weight_decay"])

    # scaler fit on TRAIN only
    scaler = fit_scaler_on_train(X_train)
    X_train_s = apply_scaler(scaler, X_train)
    X_val_s   = apply_scaler(scaler, X_val)
    X_test_s  = apply_scaler(scaler, X_test)

    train_loader = DataLoader(SeqDataset(X_train_s, y_train), batch_size=batch_size, shuffle=False)
    val_loader   = DataLoader(SeqDataset(X_val_s,   y_val),   batch_size=batch_size, shuffle=False)

    model = build_model().to(DEVICE)
    _ = train_with_early_stopping(model, train_loader, val_loader, lr=lr, weight_decay=wd)

    # test untouched until here
    y_pred = predict_model(model, X_test_s, batch_size=batch_size)

    final_rmse = rmse_np(y_test, y_pred)
    naive_rmse = rmse_np(y_test, np.zeros_like(y_test))

    pred_signs   = np.sign(y_pred)
    actual_signs = np.sign(y_test)
    valid_idx = actual_signs != 0
    dir_acc = accuracy_score(actual_signs[valid_idx], pred_signs[valid_idx])

    print(f"\n=== {model_name} RESULTS ===")
    print(f"Model RMSE:   {final_rmse:.6f}")
    print(f"Baseline RMSE:{naive_rmse:.6f}")
    print(f"Improvement:  {((naive_rmse - final_rmse)/naive_rmse)*100:.2f}%")
    print(f"Directional Accuracy: {dir_acc*100:.2f}%")

    # same output-style graph
    plt.figure(figsize=(12, 6))
    plt.plot(y_test[:100], label="Actual Change", color="black", alpha=0.5)
    plt.plot(y_pred[:100], label=f"{model_name} Pred", linestyle="--")
    plt.axhline(0, lw=0.5)
    plt.title(f"{model_name}: ATM IV Change Prediction (Delta)")
    plt.legend()
    plt.show()

    # NN "feature importance" analogue
    _, imps = permutation_importance_seq(model, X_test_s, y_test, feature_cols, batch_size=batch_size)
    plot_feature_importance(imps, feature_cols, title=f"{model_name} Feature Importance (Permutation ΔRMSE)")

    return final_rmse, dir_acc

# ---- Final RNN ----
bestRNN = study_rnn.best_params
def make_final_rnn():
    return RNNRegressor(
        n_features=n_features,
        hidden_size=int(bestRNN["hidden_size"]),
        num_layers=int(bestRNN["num_layers"]),
        dropout=float(bestRNN["dropout"]),
        rnn_type=str(bestRNN["rnn_type"])
    )

rmse_rnn, dir_rnn = evaluate_final("RNN", make_final_rnn, bestRNN)

# ---- Final CNN ----
bestCNN = study_cnn.best_params
def make_final_cnn():
    return CNN1DRegressor(
        n_features=n_features,
        channels=int(bestCNN["channels"]),
        kernel_size=int(bestCNN["kernel_size"]),
        dropout=float(bestCNN["dropout"])
    )

rmse_cnn, dir_cnn = evaluate_final("CNN", make_final_cnn, bestCNN)

print("\n>>> Summary")
print(f"RNN  RMSE={rmse_rnn:.6f} | DirAcc={dir_rnn*100:.2f}%")
print(f"CNN  RMSE={rmse_cnn:.6f} | DirAcc={dir_cnn*100:.2f}%")
